In [1]:
# import cv2
# import numpy as np
# from yolo_predictions import YOLO_Pred  # Ensure this is the correct path to your YOLO_Pred class
# from collections import deque
# import math

In [6]:
# onnx_model = './Model-20240613T072338Z-001/Model/weights/best.onnx'  # Adjust the model path if needed
# data_yaml = 'data.yaml'
# yolo = YOLO_Pred(onnx_model, data_yaml)

In [2]:
# class VehicleTracker:
#     def __init__(self, onnx_model, data_yaml):
#         self.yolo_pred = YOLO_Pred(onnx_model, data_yaml)
#         self.trackers = []
#         self.vehicle_count = {"car": 0, "bike": 0}
#         self.line_y = 400

#     def track_vehicles(self, frame):
#         frame, detections = self.yolo_pred.predictions(frame)

#         # Initialize trackers for new detections
#         for class_name, bbox in detections:
#             if class_name in ["car", "bike"]:
#                 tracker = cv2.TrackerCSRT_create()
#                 tracker.init(frame, bbox)
#                 self.trackers.append((tracker, class_name)) 

#         # Update trackers and count vehicles
#         new_trackers = []
#         for tracker, class_name in self.trackers:
#             success, bbox = tracker.update(frame)
#             if success:
#                 x, y, w, h = [int(v) for v in bbox]
#                 if y < self.line_y < y + h:
#                     self.vehicle_count[class_name] += 1
#                 else:
#                     new_trackers.append((tracker, class_name))
#                 cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
#                 cv2.putText(frame, class_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
        
#         self.trackers = new_trackers

#         # Draw the ROI line
#         cv2.line(frame, (0, self.line_y), (frame.shape[1], self.line_y), (255, 0, 0), 2)
#         cv2.putText(frame, f"Cars: {self.vehicle_count['car']}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
#         cv2.putText(frame, f"Bikes: {self.vehicle_count['bike']}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

#         return frame           

In [3]:
# def main():
#     onnx_model = "./Model-20240613T072338Z-001/Model/weights/best.onnx"  # Path to your ONNX model
#     data_yaml = "data.yaml"     # Path to your YAML file

#     vehicle_tracker = VehicleTracker(onnx_model, data_yaml)

#     cap = cv2.VideoCapture("2103099-uhd_3840_2160_30fps.mp4")  # Path to your video file

#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         frame = vehicle_tracker.track_vehicles(frame)
#         cv2.imshow("Vehicle Tracking", frame)

#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()

# if __name__ == "__main__":
#     main()

ValueError: too many values to unpack (expected 2)

In [7]:
#!/usr/bin/env python
# coding: utf-8
import cv2
import numpy as np
from yolo_predictions import YOLO_Pred  # Assuming your prediction file is named yolo_pred.py and contains YOLO_Pred class

def initialize_tracker(tracker_type):
    if tracker_type == 'BOOSTING':
        return cv2.TrackerBoosting_create()
    elif tracker_type == 'MIL':
        return cv2.TrackerMIL_create()
    elif tracker_type == 'KCF':
        return cv2.TrackerKCF_create()
    elif tracker_type == 'TLD':
        return cv2.TrackerTLD_create()
    elif tracker_type == 'MEDIANFLOW':
        return cv2.TrackerMedianFlow_create()
    elif tracker_type == 'GOTURN':
        return cv2.TrackerGOTURN_create()
    elif tracker_type == 'MOSSE':
        return cv2.TrackerMOSSE_create()
    elif tracker_type == 'CSRT':
        return cv2.TrackerCSRT_create()
    else:
        raise ValueError(f'Unknown tracker type: {tracker_type}')

def main(video_path, onnx_model, data_yaml, tracker_type='CSRT'):
    yolo_predictor = YOLO_Pred(onnx_model, data_yaml)
    cap = cv2.VideoCapture(video_path)
    trackers = cv2.MultiTracker()
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        if len(trackers.getObjects()) == 0:
            processed_frame, detections = yolo_predictor.predictions(frame)
            for detection in detections:
                x, y, w, h, class_id = detection
                tracker = initialize_tracker(tracker_type)
                trackers.add(tracker, frame, (x, y, w, h))
        else:
            success, boxes = trackers.update(frame)
            for i, box in enumerate(boxes):
                x, y, w, h = [int(v) for v in box]
                class_id = detections[i][4]  # assuming order of detections is same as trackers
                color = yolo_predictor.generate_colors(class_id)
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                class_name = yolo_predictor.labels[class_id]
                cv2.putText(frame, class_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        cv2.imshow('Vehicle Tracking', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path = '2103099-uhd_3840_2160_30fps.mp4'
    onnx_model = './Model-20240613T072338Z-001/Model/weights/best.onnx'
    data_yaml = 'data.yaml'
    tracker_type = 'CSRT'  # You can change this to any supported tracker type
    main(video_path, onnx_model, data_yaml, tracker_type)


AttributeError: module 'cv2' has no attribute 'MultiTracker'